In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Missing').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/16 07:47:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
training_data = spark.read.csv('test1.csv', header=True, inferSchema=True)

In [4]:
training_data.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [5]:
training_data.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [6]:
training_data.columns

['Name', 'age', 'Experience', 'Salary']

In [10]:
#Creating a new data to be used for training the model
from pyspark.ml.feature import VectorAssembler
featureAssembler = VectorAssembler(inputCols=["age", "Experience"], outputCol="Input Features")

In [12]:
output = featureAssembler.transform(training_data)

In [13]:
output.show()

+---------+---+----------+------+--------------+
|     Name|age|Experience|Salary|Input Features|
+---------+---+----------+------+--------------+
|    Krish| 31|        10| 30000|   [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|    [30.0,8.0]|
|    Sunny| 29|         4| 20000|    [29.0,4.0]|
|     Paul| 24|         3| 20000|    [24.0,3.0]|
|   Harsha| 21|         1| 15000|    [21.0,1.0]|
|  Shubham| 23|         2| 18000|    [23.0,2.0]|
+---------+---+----------+------+--------------+



In [14]:
finalized_output = output.select('Input Features', 'Salary')

In [16]:
finalized_output.show()

+--------------+------+
|Input Features|Salary|
+--------------+------+
|   [31.0,10.0]| 30000|
|    [30.0,8.0]| 25000|
|    [29.0,4.0]| 20000|
|    [24.0,3.0]| 20000|
|    [21.0,1.0]| 15000|
|    [23.0,2.0]| 18000|
+--------------+------+



In [17]:
##Creating a test split
from pyspark.ml.regression import LinearRegression
train_data, test_data = finalized_output.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol='Input Features', labelCol='Salary').fit(train_data)

23/04/16 08:16:38 WARN Instrumentation: [0c66a211] regParam is zero, which might cause numerical instability and overfitting.
23/04/16 08:16:39 WARN InstanceBuilder$JavaBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
23/04/16 08:16:39 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/04/16 08:16:39 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
23/04/16 08:16:39 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [18]:
#Coefficients
regressor.coefficients

DenseVector([109.3058, 1199.4092])

In [19]:
#Intercepts
regressor.intercept

12187.592319054227

In [20]:
#Prediction
pred_results=regressor.evaluate(test_data)

In [22]:
pred_results.predictions.show()

+--------------+------+------------------+
|Input Features|Salary|        prediction|
+--------------+------+------------------+
|    [24.0,3.0]| 20000| 18409.15805022155|
|   [31.0,10.0]| 30000|27570.162481536143|
+--------------+------+------------------+



In [24]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

(2010.339734121153, 4217444.237654746)